In [22]:
import ee
ee.Authenticate()
ee.Initialize()

In [23]:
from datetime import timedelta
from datetime import datetime
import pandas as pd
import os
import csv

In [24]:
#siteList=["projects/ee-modis250/assets/validationGBOV/COPERNICUS_GBOV_RM7_20220616174529"]
siteList=["projects/ee-modis250/assets/validationGBOV/NEON_2013-22_RMs_updated_parsed"]

In [25]:
datapath='C:/Djamai_Najib/1_vegetation_parameters/1_scripts/LEAF-Landsat-Validation-paper/data/'

In [26]:
def exel_to_klm(xfn,name_col=3,desc_col=4,lat_col=1,lon_col=2,z_col=0):
    if not xfn.endswith('.csv'):
        print("Enter csv file")
    else:
        data = csv.reader(open(xfn), delimiter = ',')
        #Skip the 1st header row.
        next(data)
        print(data)
        #Open the file to be written.
        klm_file=xfn.replace('.csv','.kml')
        f = open(klm_file, 'w')
        
        #Writing the kml file.
        f.write("<?xml version='1.0' encoding='UTF-8'?>\n")
        f.write("<kml xmlns='http://earth.google.com/kml/2.1'>\n")
        f.write("<Document>\n")
        f.write("   <name>" + ofn + '.kml' +"</name>\n")
        for row in data:
            f.write("   <Placemark>\n")
            f.write("       <name>" + str(row[name_col]) + "</name>\n")
            f.write("       <description>" + str(row[desc_col]) + "</description>\n")
            f.write("       <Point>\n")
            f.write("           <coordinates>" + str(row[lon_col]) + "," + str(row[lat_col]) + "," + "0" + "</coordinates>\n")
            f.write("       </Point>\n")
            f.write("   </Placemark>\n")
        f.write("</Document>\n")
        f.write("</kml>\n")
        print ("File Created. ")
        return klm_file

In [46]:
for input in siteList:   
    #Convert the feature collection to a list so we can apply SL2P on features in sequence to avoid time outs on GEE
    sampleRecords =  ee.FeatureCollection(input).sort('system:time_start', False).map(lambda feature: feature.set('timeStart',feature.get('system:time_start')))
    sampleRecords =  sampleRecords.toList(sampleRecords.size())
    site=sampleRecords.getInfo()
    df=pd.DataFrame(site)
    date=[]
    lat=[]
    lon=[]
    site_name=[]
    for ii in range(len(site)):
        lon.append(site[ii]['geometry']['coordinates'][0])
        lat.append(site[ii]['geometry']['coordinates'][1]) 
        try:
            site_name.append(site[ii]['properties']['Site'])
        except:
            site_name.append(site[ii]['properties']['NEONSITE'])     
        time_start=site[ii]['properties']['system:time_start']
        if isinstance(time_start, int):
            date.append(datetime.fromtimestamp(site[ii]['properties']['system:time_start']/1000).strftime("%m/%d/%Y"))
        else:
            date.append(site[ii]['properties']['system:time_start'])    
        
    df['lat']  = lat
    df['lon']  = lon
    df['site_name']  = site_name
    df['date']  = date
    df=df[['lat','lon','site_name','date']]
    ofn=os.path.join(datapath,os.path.basename(input)+'.csv')
    print(ofn)
    df.to_csv(ofn, sep=',')
    exel_to_klm(ofn,name_col=2,desc_col=3,lat_col=0,lon_col=1,z_col=0)

C:/Djamai_Najib/1_vegetation_parameters/1_scripts/LEAF-Landsat-Validation-paper/data/NEON_2013-22_RMs_updated_parsed.csv
File Created. 


In [47]:
exel_to_klm(ofn,name_col=3,desc_col=4,lat_col=1,lon_col=2,z_col=0)

File Created. 


'C:/Djamai_Najib/1_vegetation_parameters/1_scripts/LEAF-Landsat-Validation-paper/data/NEON_2013-22_RMs_updated_parsed.kml'